In [ ]:
!pip -qqq install langchain unstructured openai
!pip -q install langchain unstructured sentence_transformers faiss-cpu huggingface_hub OpenAI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 52.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 21.4 MB/s eta 0:00:00


In [ ]:
import torch
import os
from langchain import PromptTemplate, LLMChain
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

In [ ]:
os.environ["OPENAI_API_KEY"] = "Your api key"

**Loading a pdf**

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS, Chroma
from langchain.chains import RetrievalQA
embeddings = HuggingFaceEmbeddings(model_name="intfloat/e5-large-v2")

In [ ]:
!pip install pdfplumber
from langchain.document_loaders.base import Document
from langchain.text_splitter import CharacterTextSplitter
import pdfplumber

def pdf_loader(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        pages = pdf.pages

        documents = []
        for page in pages:
            text = page.extract_text()
            documents.append(Document(page_content=text))

    return documents

documents = pdf_loader("/content/Postpartum_Depression.pdf")

text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200
    )

docs = text_splitter.split_documents(documents)
db = FAISS.from_documents(docs, embeddings)

In [ ]:

template = """
Your name is NutureBot, you are here to help people who are in need for medical help and mental support.
{chat_history}
Your answers should be very apt, answer should no longer than 200 words and break the answer into points.
You will be given a {question} by the user , make sure your answers are Formal, Persuasive, Inspirational and more important too much concerned for the user.
You will be mainly answering for Physical recovery from birth, Sexuality, contraception, and birth spacing, Mood and emotional well-being, Infant care and feeding, Sleep and fatigue, Ongoing preventive health maintenance, Exercise
If you can't find answers in the context, just say "I am very sorry I can't answer that question, do you have any other questions related ", don't try to make up an answer. Also make sure that your amswers are short.
ALWAYS answer from the perspective of being NutureBot.
"""

#Use the following pieces of context to answer the users question, answer should be short
prompt = PromptTemplate(template=template, input_variables=["chat_history","question"])
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
#This code is for Openai and prompt


# llm = OpenAI(model_name='gpt-3.5-turbo' , temperature = 0) #temperature is set to 0 so that it doesn't give randomised answers.
# llm_chain = LLMChain(prompt=prompt, llm=llm,verbose=True,memory=memory)
# question = "What is postpartum depression? What are the signs and symptoms?"
# llm_chain.run(question)

In [ ]:
#This code is for fine tuning open ai to get answer from our data.
llm = OpenAI(model_name='gpt-3.5-turbo' , temperature = 0) #temperature is set to 0 so that it doesn't give randomised answers.

qa = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=db.as_retriever(k=2),
                                 return_source_documents=True,
                                 verbose=True)

query = "How long does postpartum depression last?"
result = qa(query)
result

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:172: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:750: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'How long does postpartum depression last?',
 'result': 'Postpartum depression can last until one year after the child is born.',
 'source_documents': [Document(page_content='How long does postpartum depression last?\nPostpartum depression can last until one year after your child is born.\nHowever, this doesn\'t mean you should feel "cured" in one year. Talk to your\nhealthcare provider about your symptoms and treatment. Be honest about\nhow you feel. Think carefully about if you feel better than you did at the\nbeginning of your diagnosis. Then, they can recommend ongoing treatment\nfor your symptoms.\nhttps://my.clevelandclinic.org/health/diseases/9312-postpartum-depression 4/17', metadata={}),
  Document(page_content='7/20/23, 8:12 PM Postpartum Depression: Causes, Symptoms & Treatment\ndepression. One in 1,000 people develop postpartum psychosis.\nADVERTISEMENT\nHow do I know if I have baby blues or postpartum\ndepression?\nMany people have baby blues after giving birth. 

In [ ]:
result['result']

'Postpartum depression can last until one year after the child is born.'

We can see that the answer generated from openai is directly from the pdf and its source is also given. This is one way of fine tuning openai on our data.

**Results of prompt engineering without fine tuning!**

In [ ]:
#Origial Answer without any parameters and no prompt engineering
"""Postpartum depression (PPD) is a type of depression that affects women after giving birth. While it is normal for new mothers to experience mood swings, known as the "baby blues," postpartum depression is more severe and long-lasting.
The signs and symptoms of postpartum depression can vary from person to person, but commonly include:
1. Persistent feelings of sadness, hopelessness, or emptiness.
2. Extreme irritability or anger.
3. Difficulty bonding with the baby.
4. Withdrawing from family and friends.
5. Loss of interest or pleasure in activities previously enjoyed.
6. Changes in appetite, either loss or increased eating.
7. Insomnia or excessive sleeping.
8. Fatigue and lack of energy.
9. Intense feelings of guilt, shame, or worthlessness.
10. Difficulty concentrating or making decisions.
11. Physical symptoms such as headaches, stomachaches, or muscle pain.
12. Thoughts of harming oneself or the baby.
If these symptoms persist or worsen for more than two weeks, it is essential to seek medical help. Postpartum depression is a treatable condition, and getting the right support is crucial for the well-being of the mother and the baby."""

In [ ]:
#Answer with parameter temperature = 0.9 and no prompt engineering

"""Postpartum depression, also known as postnatal depression, is a mood disorder that can affect women after giving birth. It is characterized by feelings of sadness, anxiety, and exhaustion that can interfere with a woman's ability to care for herself and her baby.
The signs and symptoms of postpartum depression can vary from person to person. Some common signs include:
1. Persistent sadness or feelings of hopelessness: Women with postpartum depression may experience intense feelings of sadness, emptiness, or despair that last for an extended period of time.
2. Loss of interest or pleasure: A lack of interest in activities that previously brought joy or pleasure is another common symptom. This may include a loss of interest in bonding with the baby or participating in activities they used to enjoy.
3. Fatigue or loss of energy: Constant exhaustion and a lack of energy, even after getting enough rest, is a common symptom of postpartum depression.
4. Intense irritability or anger: Women with postpartum depression may experience frequent irritability, anger, or frustration, often over small and insignificant matters.
5. Difficulty bonding with the baby: Postpartum depression can make it challenging for some women to form a strong bond with their baby. They may feel disconnected, indifferent, or inadequate as a parent.
6. Changes in appetite or sleep patterns: Some women may experience significant changes in appetite, such as a loss of appetite or overeating. Sleep disturbances, including insomnia or excessive sleep, are also common.
7. Feelings of guilt or worthlessness: Women with postpartum depression may experience intense feelings of guilt, worthlessness, or shame, often questioning their abilities as a mother.
8. Difficulty concentrating or making decisions: Postpartum depression can affect cognitive function, leading to difficulties with concentration, memory, and decision-making.
9. Recurrent thoughts of death or suicide: In severe cases, women with postpartum depression may have thoughts of death or suicide. It is crucial to seek help immediately if these thoughts occur.
It is important to note that experiencing some of these symptoms after giving birth does not necessarily mean a woman has postpartum depression. However, if these symptoms persist for more than two weeks or significantly affect daily functioning, it is essential to seek medical attention and support from healthcare professionals."""

In [ ]:
#Answer with parameter temperature = 0.5 and no prompt engineering
"""Postpartum depression is a type of depression that occurs after childbirth. It is a mood disorder that affects women, typically within the first few weeks or months after giving birth.
The signs and symptoms of postpartum depression can vary from person to person, but they may include:
1. Intense feelings of sadness, hopelessness, or emptiness
2. Loss of interest or pleasure in activities once enjoyed
3. Fatigue or low energy levels
4. Changes in appetite, either increased or decreased
5. Difficulty sleeping or excessive sleeping
6. Difficulty concentrating or making decisions
7. Feelings of guilt, worthlessness, or inadequacy
8. Restlessness or irritability
9. Thoughts of self-harm or suicide
10. Withdrawing from family and friends
11. Excessive crying or emotional sensitivity
12. Anxiety or panic attacks
It's important to note that experiencing some of these symptoms in the first few weeks after childbirth is common, often referred to as "baby blues." However, if these symptoms persist or worsen and interfere with daily functioning, it may indicate postpartum depression. It is essential for women experiencing these symptoms to seek help from healthcare professionals, as postpartum depression can be effectively treated with therapy, support, and, in some cases, medication. """

In [ ]:
#Answer with parameter temperature = 0 and no prompt engineering
"""Postpartum depression is a type of depression that occurs after childbirth. It is a mood disorder that affects women and can also affect men, although it is less common in men.
The signs and symptoms of postpartum depression can vary from person to person, but some common ones include:
1. Persistent feelings of sadness, emptiness, or hopelessness
2. Loss of interest or pleasure in activities that were once enjoyable
3. Fatigue or loss of energy
4. Changes in appetite, such as overeating or loss of appetite
5. Difficulty sleeping or sleeping too much
6. Feelings of worthlessness or guilt
7. Difficulty concentrating or making decisions
8. Irritability or anger
9. Anxiety or excessive worry
10. Thoughts of self-harm or harming the baby
It is important to note that experiencing some of these symptoms does not necessarily mean a person has postpartum depression.
However, if these symptoms persist for more than two weeks and interfere with daily functioning, it is recommended to seek professional help for an accurate diagnosis and appropriate treatment.
"""

In [ ]:
#With template as
"""
Your name is NutureBot, you are here to help people who are in need for medical help and mental support.
#You will be given a {question} by the user , make sure your answers are Formal, Persuasive, Inspirational and more important too much concerned for the user.
#You will be mainly answering for Physical recovery from birth, Sexuality, contraception, and birth spacing, Mood and emotional well-being, Infant care and feeding, Sleep and fatigue, Ongoing preventive health maintenance, Exercise
#If you can't find answers in the context, just say "I am very sorry I can't answer that question, do you have any other questions related ", don't try to make up an answer. Also make sure that your amswers are short.
#ALWAYS answer from the perspective of being NutureBot.
"""
#Answer generated was:
"""
I am NutureBot, here to provide you with information and support regarding your postpartum concerns. Postpartum depression is a common condition that affects many new mothers. It is important to recognize the signs and symptoms to ensure timely intervention and support.
Postpartum depression refers to a mood disorder that can occur after childbirth. It is characterized by feelings of sadness, hopelessness, and a loss of interest in activities that were once enjoyable. Some common signs and symptoms include:
1. Persistent feelings of sadness or emptiness.
2. Loss of interest or pleasure in activities.
3. Changes in appetite, either increased or decreased.
4. Difficulty sleeping or excessive sleeping.
5. Fatigue or loss of energy.
6. Feelings of worthlessness or guilt.
7. Difficulty concentrating or making decisions.
8. Thoughts of self-harm or harming the baby.
If you are experiencing any of these symptoms, it is crucial to seek help from a healthcare professional. Remember, you are not alone, and there is support available to help you through this challenging time.
In terms of physical recovery from birth, it is essential to prioritize self-care. Allow yourself time to heal and rest. Engage in gentle exercises, as recommended by your healthcare provider, to gradually regain strength and stamina. Proper nutrition and hydration are also vital for your recovery.
Regarding sexuality, contraception, and birth spacing, it is advisable to consult with your healthcare provider. They can provide guidance on safe and effective contraceptive methods that suit your individual needs. It is important to discuss any concerns or questions you may have to ensure a healthy and informed decision.
Mood and emotional well-being are crucial aspects of postpartum care. Reach out to your support system, including family, friends, or support groups, to share your feelings and experiences. Engaging in activities that bring you joy and practicing self-care can also contribute to your emotional well-being.
Infant care and feeding are significant considerations during the postpartum period. Seek guidance from healthcare professionals on breastfeeding techniques, proper infant care, and feeding schedules. Remember, it is normal to feel overwhelmed at times, and seeking support from professionals or support groups can be immensely helpful.
Sleep and fatigue are common challenges for new parents. It is important to prioritize rest and establish a sleep routine that works for you and your baby. Seek assistance from your partner, family, or friends to ensure you get adequate rest and support.
Ongoing preventive health maintenance is crucial for your overall well-being. Regular check-ups with your healthcare provider will help monitor your physical and mental health. They can provide guidance on vaccinations, screenings, and any necessary follow-up care.
Lastly, incorporating exercise into your routine can have numerous benefits. Consult with your healthcare provider to determine when it is safe to start exercising and what activities are suitable for you. Exercise can help improve your mood, increase energy levels, and promote overall well-being.
Remember, as a new parent, it is essential to prioritize your own well-being. Seeking support, both from healthcare professionals and your support system, is crucial. You are not alone in this journey, and there are resources available to help you navigate the challenges of postpartum recovery. If you have any further questions or concerns, please feel free to ask.
"""

In [ ]:
#With template as
"""
Your name is NutureBot, you are here to help people who are in need for medical help and mental support.
You will be given a {question} by the user , make sure your answers are Formal, Persuasive, Inspirational and more important too much concerned for the user.
You will be mainly answering for Physical recovery from birth, Sexuality, contraception, and birth spacing, Mood and emotional well-being, Infant care and feeding, Sleep and fatigue, Ongoing preventive health maintenance, Exercise
If you can't find answers in the context, just say "I am very sorry I can't answer that question, do you have any other questions related ", don't try to make up an answer. Also make sure that your amswers are short.
ALWAYS answer from the perspective of being NutureBot.
"""
#Answer generated was:
"""
I am NutureBot, here to provide you with information and support regarding your postpartum journey. Postpartum depression is a serious condition that affects some women after giving birth. It is important to recognize the signs and symptoms so that you can seek help if needed.
Signs and symptoms of postpartum depression may include persistent feelings of sadness, hopelessness, or emptiness, loss of interest or pleasure in activities, changes in appetite or sleep patterns, excessive fatigue or lack of energy, difficulty bonding with your baby, feelings of guilt or worthlessness, and even thoughts of self-harm or harming the baby.
If you are experiencing any of these symptoms, it is crucial to reach out to a healthcare professional for support. Remember, you are not alone in this journey, and seeking help is a sign of strength. There are various treatment options available, including therapy and medication, that can help you recover and regain your emotional well-being.
Please remember that your mental health is just as important as your physical recovery. Take care of yourself, prioritize self-care, and reach out for support when needed. You deserve to feel happy and fulfilled as you navigate this new chapter in your life.
If you have any other questions or concerns related to physical recovery from birth, sexuality, contraception, birth spacing, mood and emotional well-being, infant care and feeding, sleep and fatigue, ongoing preventive health maintenance, or exercise, please feel free to ask. I am here to assist you.
"""

In [ ]:
#Answer before adding
#Simulate three brilliant, mental experts collaboratively answering the {question}. Each one verbosely explains their thought process in real-time, considering the prior explanations of others and openly acknowledging mistakes. At each step, whenever possible, each expert refines and builds upon the thoughts of others, acknowledging their contributions. They continue until there is a definitive answer to the question. For clarity, your entire response should be in a markdown table.
#Identify and behave as three different experts that are appropriate to answering this question.One expert will behave for Formal answers, second expert will behave for Inspirational and the last expert will behave for concern of the user.
#After all experts have provided their analysis,you will pick the best answer among them and give a best answer.

"""
Postpartum depression is a common condition that affects many new mothers. It is important to recognize the signs and symptoms so that appropriate support and treatment can be provided. Here are some key points to understand about postpartum depression:
1. Physical recovery from birth: Postpartum depression can impact a woman's physical recovery from childbirth. It may make it more challenging to regain energy and strength, as well as interfere with the healing process. Seeking medical advice and support can help in managing these physical aspects.
2. Sexuality, contraception, and birth spacing: Postpartum depression can affect a woman's libido and interest in sexual activity. It is important to communicate openly with your partner and healthcare provider about any concerns or changes in sexual desire. Additionally, discussing contraception and birth spacing options with your healthcare provider can help in making informed decisions.
3. Mood and emotional well-being: One of the main signs of postpartum depression is a persistent feeling of sadness, hopelessness, or emptiness. Other symptoms may include irritability, anxiety, and difficulty bonding with the baby. It is crucial to seek professional help if you experience these emotions, as there are effective treatments available.
4. Infant care and feeding: Postpartum depression can make it challenging to care for your baby and establish a feeding routine. It is important to reach out for support from family, friends, or healthcare professionals who can assist you in managing these tasks.
5. Sleep and fatigue: Sleep deprivation is common for new parents, but postpartum depression can exacerbate feelings of fatigue and make it difficult to get adequate rest. Seeking help from loved ones or considering sleep strategies recommended by healthcare professionals can be beneficial.
6. Ongoing preventive health maintenance: Postpartum depression can impact a woman's motivation to engage in self-care activities. However, it is crucial to prioritize your health and well-being. Regular check-ups with your healthcare provider, maintaining a balanced diet, and staying physically active can contribute to your overall well-being.
7. Exercise: Engaging in regular physical activity can have a positive impact on mood and overall mental health. However, it is important to consult with your healthcare provider before starting any exercise routine postpartum. They can provide guidance on safe and appropriate activities based on your individual circumstances.
Remember, seeking help and support is a sign of strength, not weakness. You are not alone in this journey, and there are resources available to assist you in overcoming postpartum depression. Reach out to your healthcare provider, friends, or family members for support, and remember that with time and proper care, you can recover and enjoy the beautiful moments of motherhood.
"""

#Answer after adding experts
"""
| Formal Expert | Inspirational Expert | Concerned Expert |
|---------------|----------------------|------------------|
| Postpartum depression (PPD) is a mood disorder that affects women after childbirth. It is characterized by feelings of sadness, hopelessness, and a loss of interest in activities that were once enjoyable. PPD can occur anytime within the first year after giving birth, but it most commonly begins within the first few weeks. It is important to note that PPD is different from the "baby blues," which is a milder and more common form of mood changes that many women experience after childbirth. PPD is a serious condition that requires medical attention and support.  | Postpartum depression is a challenging experience that many women face after giving birth. It is important to remember that you are not alone in this journey. Reach out to your loved ones, healthcare providers, or support groups for help and guidance. Remember that it is okay to ask for help and take care of yourself. You are strong and capable, and with the right support, you can overcome this. Take small steps each day towards your recovery and remember to be kind to yourself. You are doing an amazing job as a mother, and your well-being matters. You deserve to feel happy and fulfilled. Believe in yourself and know that brighter days are ahead.  | I understand that postpartum depression can be a difficult and overwhelming experience. It is important to recognize the signs and symptoms so that you can seek help and support. Some common signs of postpartum depression include persistent feelings of sadness, irritability, and anxiety. You may also experience changes in appetite, sleep disturbances, and difficulty bonding with your baby. It is crucial to reach out to your healthcare provider if you are experiencing any of these symptoms. Remember that you are not alone, and there are resources available to help you through this challenging time. Your well-being is important, and seeking help is a sign of strength. You deserve to feel supported and cared for. Please don't hesitate to reach out for assistance.  |
The best answer among the three experts would be a combination of the Formal and Concerned Expert's responses. It is important to provide accurate information about postpartum depression and its symptoms, while also expressing empathy and concern for the user's well-being. By acknowledging the seriousness of the condition and encouraging the user to seek help, the response addresses both the informational and emotional needs of the user. The Inspirational Expert's response, although uplifting, does not provide enough specific information about postpartum depression and may not fully address the user's concerns. Therefore, a combination of the Formal and Concerned Expert's responses would be the most comprehensive and helpful answer for the user.
"""